## A simple Chatbot

### Loading the model

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

ChatModels are the instances of LangChain `Runnables`, they expose a standard interface for interacting with them.
We can simple use `invoke` method to call the model.

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([
    HumanMessage(content="Hi I am Manoj")
])

AIMessage(content='Hi Manoj! Nice to meet you! Is there something I can help you with or would you like to chat?', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 15, 'total_tokens': 40, 'completion_time': 0.019154745, 'prompt_time': 0.00309007, 'queue_time': None, 'total_time': 0.022244815}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-da85ea29-5c07-45f9-a6b1-dedc524f5ce9-0')

In [5]:
model.invoke([
    HumanMessage(content="What is my name?")
])

AIMessage(content="I apologize, but I don't know your name. I'm a large language model, I don't have the ability to know or retain personal information about individuals. Each time you interact with me, it's a new conversation and I don't have any prior knowledge or context about you. If you'd like to share your name with me, I'd be happy to learn it!", response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 15, 'total_tokens': 93, 'completion_time': 0.061547363, 'prompt_time': 0.003444526, 'queue_time': None, 'total_time': 0.064991889}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_f128cfcd6c', 'finish_reason': 'stop', 'logprobs': None}, id='run-36b7bc5a-1b9c-4ed1-bd08-d775cebefe03-0')

We can see that it doesnt take the previous conversation turn into context and cannot answer the qestion. This is not how a chatbot should work.

To get around this we need to pass the entire conversation history into the model.

In [6]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi Iam Manoj"),
    AIMessage(content="Hello Manoj! How can I assist you today?"),
    HumanMessage(content="What is my name?")
])

AIMessage(content='Your name is Manoj!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 41, 'total_tokens': 48, 'completion_time': 0.004798075, 'prompt_time': 0.006930433, 'queue_time': None, 'total_time': 0.011728507999999999}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-37943557-c7d9-44c8-9211-ee3edd6b55a2-0')

Now we get a good response.
This is the basic underpinning a chatbot's ability to interact conversationally.

### Message History
We can use a Message History class to wrap our model and can make it stateful which will keep track of inputs and outputs of the model and store them in some datastore.

Any future interactions will then load those messages and pass them into the chain as part of the input.

**Langchain_community** must be installed first

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# session_id: This session_id is used to distinguish between separate conversaitons,

# will return a Message History object
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    model,
    get_session_history
)

In [8]:
# we need to create a config that we pass into the runnable every time.

config = {
    "configurable": {
        "session_id": "111"
    }
}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi I am Manoj")],
    config=config
)

response.content

Parent run 7db3aebf-5f05-405a-b947-5e8069cb1f59 not found for run 4cb79d73-e927-49d8-ae5b-5a78647528a9. Treating as a root run.


"Hi Manoj! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [10]:
# Now again running with the same config which contain the same session id so that we are in the same conversation,

response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

response.content

Parent run 4b852790-7bfe-4a7c-a224-a16b3b66208f not found for run 4dd2ab63-1c94-4751-807b-37c649907a48. Treating as a root run.


'Your name is Manoj!'

In [11]:
# If we change the session id then it will be a new conversation,

response = with_message_history.invoke(
    [HumanMessage(content="Hey what is my name?")],
    config={
        "configurable": {
            "session_id": "222"
        }
    }
)

response.content

Parent run e6ac75bc-8f1b-48f8-baa8-0d4c020c1091 not found for run 712d9c53-bc17-41af-b669-f7b20bb3d8bf. Treating as a root run.


"I apologize, but I'm a large language model, I don't have the ability to know your name or any personal information about you. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to know it and address you by name!"

In [12]:
type(store)

dict

In [13]:
store

{'111': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am Manoj'), AIMessage(content="Hi Manoj! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 15, 'total_tokens': 42, 'completion_time': 0.020707135, 'prompt_time': 0.003414331, 'queue_time': None, 'total_time': 0.024121466}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cb79d73-e927-49d8-ae5b-5a78647528a9-0'), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Manoj!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 56, 'total_tokens': 63, 'completion_time': 0.004812692, 'prompt_time': 0.00917908, 'queue_time': None, 'total_time': 0.013991772}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_f128cfcd6c', 'finish_reason': 'stop', 'logprobs': None}, id='run-4dd2ab63-

In [14]:
store["111"].messages

[HumanMessage(content='Hi I am Manoj'),
 AIMessage(content="Hi Manoj! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 15, 'total_tokens': 42, 'completion_time': 0.020707135, 'prompt_time': 0.003414331, 'queue_time': None, 'total_time': 0.024121466}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cb79d73-e927-49d8-ae5b-5a78647528a9-0'),
 HumanMessage(content='What is my name?'),
 AIMessage(content='Your name is Manoj!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 56, 'total_tokens': 63, 'completion_time': 0.004812692, 'prompt_time': 0.00917908, 'queue_time': None, 'total_time': 0.013991772}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_f128cfcd6c', 'finish_reason': 'stop', 'logprobs': None}, id='run-4dd2ab63-1c94-4751-807b-37c649907a48-0')]

### We can make more complicated and personalized chatbot by adding in a prompt template.

**Prompt Templates**
Prompt Templates help to turn raw user information into a format that the LLM can work with.(message)

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        # messagesplaceholder to pass all the messages in a dictionary with a "messages" key where that contains a list of messages.
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
response = chain.invoke(
    {
        "messages": [HumanMessage(content="Hello  I am Manoj Baniya.")]
    }
)

response.content

"Nice to meet you, Manoj Baniya! I'm happy to be your helpful assistant. How can I assist you today? Do you have a specific question, topic you'd like to discuss, or task you'd like help with? I'm all ears!"

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history
)

In [18]:
config = {
    "configurable": {
        "session_id": "new_session"
    }
}

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello I am Manoj Baniya.")],
    config=config
)

Parent run 5e8d7aca-cc1f-499a-9b5d-4355bb1aba70 not found for run c3b00dfd-7fa6-4a28-8b58-2939afb916c4. Treating as a root run.


In [20]:
print(response.content)

Nice to meet you, Manoj Baniya! How can I assist you today? Do you have a specific question, topic you'd like to discuss, or perhaps a task you'd like help with? I'm all ears!


### A bit more complicated Prompt Template

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 1: for the system
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language} language."
        ),
        # 2: for the user
        # All the history messages will be here in the "messages" in dict format which will contain a list of messages.
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
# **language** is a variable that should be passed in the messages when calling

response = chain.invoke(
    {
        "messages": [
            HumanMessage(content="Hello my name is Manoj Baniya, I am from Nepal.")
        ],
        "language": "Hinglish"
    }
)

response.content

"Namaste Manoj Baniya ji! Welcome, I'm glad to meet you. You're from Nepal, right? How can I help you today? Kya problem hai?"

In [23]:
response = chain.invoke(
    {
        "messages": [
            HumanMessage(content="Hello what is my name?")
        ],
        "language": "Hinglish"
    }
)

response.content

"Aapka naam kya hai? Sorry, I didn't understand that. You didn't tell me your name, so I can't tell you what it is. Can you please introduce yourself?"

Now finally wrapping this new prompt template with input variable in the MessageHistory to keep memory in the chatbot.

In [24]:
with_message_history = RunnableWithMessageHistory(
    chain, # which is the prompt | model
    get_session_history, # function to get the session history
    input_messages_key="messages", # key to pass the messages
)

In [25]:
config = {
    "configurable": {
        "session_id": "333"
    }
}

In [26]:
response = with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="Hello my name is Manoj Baniya and I am from Nepal.")
        ],
        "language": "Hinglish"
    },
    config=config
)

response.content

Parent run 6bb890a0-0705-457d-bae5-e8fdb050329a not found for run 1c9826d7-b305-4bc9-8cf8-2facf43bbc08. Treating as a root run.


"Namaste Manoj Baniya ji! Aapka swagat hai! (Hello Manoj Baniya ji! Welcome!) Nepal se aap hai? (Are you from Nepal?) It's great to have you here! How can I assist you today?"

In [27]:
response = with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="Hey who am I?")
        ],
        "language": "Hinglish"
    },
    config=config
)

Parent run 551291db-bcea-4b87-8ff6-02e7a5fec2d1 not found for run 797961b7-add1-4dab-88b6-2f5fddd47dc3. Treating as a root run.


In [28]:
response.content

'Aap Manoj Baniya hai, Nepal se aapka naam hai! (You are Manoj Baniya, and your name is from Nepal!)'

### Important!

One important to understand well while building chatbots is to manage conversation history.

If left unmanaged the list of messages will grow `unbounded and potentially cause overflow the context window of the LLM`

So it is important to add a step that limits the size of the messages we are passing in.We do this BEFORE the prompt template but AFTER we load previous messages from Message History

We can do this by adding a simple step in front of the prompt that modifies the `messages` key appropriately, and then wrap that new chain in the Message History class

In LangChain **trim_messages** helper function is available to reduce how many messages we are sending to the model.
The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:

In [29]:
from langchain_core.messages import trim_messages, SystemMessage

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant answer the following questions to the best of your ability"),
    HumanMessage(content="hi! I'm Manoj Baniya and I am from Nepal."),
    AIMessage(content="hi! Mano I am assistant. How can I help you today?"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\acer\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled b

[SystemMessage(content="you're a good assistant answer the following questions to the best of your ability"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [30]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough


# To use trimmer in our chain, we just need to run the trimmer before we pass the messages input to our prompt.

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Hello what is my name?")],
        "language": "Hinglish"
    }
)

response.content

"I don't know your name, buddy!"

In [31]:
# but if we ask other quesions it can answer it

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="which ice cream do i like?")],
        "language": "Hinglish"
    }
)

response.content

'maine pata hai! You like vanilla ice cream!'

### Wrapping this in Message History

In [34]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {
    "configurable": {
        "session_id": "999"
    }
}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my name?")],
        "language": "Hinglish"
    },
    config=config
)

response.content

Parent run afd5c7da-9701-421e-9dc0-ed3e5ad24ee2 not found for run 2fedfb9f-3ffc-4347-a66f-4da435e24d27. Treating as a root run.


"I don't know, sorry!"

As expected, the first message where we stated our name has been trimmed. Plus there's now two new messages in the chat history (our latest question and the latest response). This means that even more information that used to be accessible in our conversation history is no longer available! In this case our initial math question has been trimmed from the history as well, so the model no longer knows about it:

Use LamgSmith trace to know what is happening under the hood here

### Streaming
This is to improve the User Experience with chatbot where each generated token is shown as it is generating so user dont have to wait long till all the tokens are generated.

In [37]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm Manoj. tell me a nice joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Parent run e4f5a96a-4636-4e9f-a843-63fcc7db2b82 not found for run 0cb33fb9-cc1e-4ccb-9a8b-b01bb5cb9c79. Treating as a root run.


|Hi| Man|oj|!| Nice| to| meet| you|!

|Here|'s| a| joke| for| you|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| smile|!| Do| you| want| to| hear| another| one|?||